# Middleware: Human In The Loop
<img src="./assets/LC_HITL.png" width="300">

First, we'll connect to our SQLite database that contains employee and music store data.


In [ ]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Now we'll create a tool that allows the agent to execute SQL queries. This is what we'll require approval for.


In [ ]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

We'll define a system prompt that guides the agent to be careful and methodical when working with the database.


In [ ]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.`

Next, we'll create an agent with **Human-in-the-Loop** middleware. Every time the agent wants to execute SQL, it will pause and wait for human approval.


In [ ]:
import * as setup from "./setup.ts";
import { createAgent, humanInTheLoopMiddleware } from "langchain";
import { MemorySaver } from "@langchain/langgraph";

const checkpointer = new MemorySaver();
const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [executeSQL],
    systemPrompt: SYSTEM,
    checkpointer,
    middleware: [humanInTheLoopMiddleware({
        interruptOn: {
            execute_sql: {
                allowedDecisions: ["approve", "reject"]
            }
        }
    })]
});

Let's test it! We'll ask about employees. The agent will pause at each SQL query, and we'll approve them by resuming with `type: "accept"`.


In [ ]:
import { red } from "ansis"
import { Command } from "@langchain/langgraph";
import type { HITLRequest, HITLResponse } from "langchain";

const config = { configurable: { thread_id: "1" } };

let result = await agent.invoke({
    messages: "What are the names of all the employees?"
}, config);

while ('__interrupt__' in result) {
    console.log(red("-".repeat(80)));
    
    // Access the HITLRequest from the interrupt
    const hitlRequest = result.__interrupt__[0].value as HITLRequest;
    
    // Display all action requests
    hitlRequest.actionRequests.forEach((actionRequest) => {
        console.log(red(actionRequest.description));
    });
    
    console.log(red("-".repeat(80)));

    // Create decisions for each action request
    const resume: HITLResponse = {
        decisions: hitlRequest.actionRequests.map(() => ({ 
            type: "approve" 
        }))
        // Or to reject:
        // decisions: hitlRequest.actionRequests.map(() => ({ 
        //     type: "reject", 
        //     message: "the database is offline." 
        // }))
    };

    result = await agent.invoke(
        new Command({ resume }),
        config
    );
}

console.log(result.messages.at(-1).content);